In [1]:
import json
import pandas as pd
import numpy as np
from skimage import io
from skimage import img_as_ubyte
from skimage import color

import io as bytes_io
import base64

In [2]:
REQUEST = json.dumps({
'path' : {},
'args' : {}
})

In [3]:
URL_DOMAIN="https://img.opf-dev.jp"
img_types=["osm",
                    "landsat8",
                    "landsat8-201808",
                    "true",
                    "natural",
                    "av2ori/band1",
                    "av2ori/band2",
                    "av2ori/band3",
                    "av2ori/band4"]


In [4]:
# 表示例
# img_type = img_types[0]
# ## 富士山の座標
# z="13"
# x="7252"
# y="3234"
# url = "{}/{}/{}/{}/{}.png".format(URL_DOMAIN, img_type, z, x, y)
# img = io.imread(url)

# 共通して使うutil系

In [6]:
def validation_check(x, y, z, img_type="osm"):
    """バリデーションチェック"""
    
    if not img_type or not x or not y or not z: #or ~x or ~y or ~z:
        print("ERROR: Input format error. please input `/img/[img_type]/[z]/[x]/[y]`")
        print("ex. http://localhost:8822/img/osm/13/7252/3234")
        return False
    
    elif img_type not in img_types:
        print("ERROR: img_type format error. please input next format`/img/[img_type]/[z]/[x]/[y]`")
        print("img_types only allows [{}]".format(", ".join(img_types)))
        return False
    
    elif not (x+y+z).isdecimal():
        print("ERROR: x/y/z format error")
        print("x/y/z only allows  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]")
        return False

    else:
        return True
    

# そのまま公開されている画像を持ってくるだけのAPI

In [7]:
# GET /img/:img_type/:z/:x/:y

# 読み取り
request = json.loads(REQUEST)

# 必要項目抽出
img_type = request['path'].get('img_type')
x = request['path'].get('x')
y = request['path'].get('y')
z = request['path'].get('z')

if validation_check(x, y, z, img_type=img_type):
    try:
        #imageをAPIからgetし、numpyに変換
        img_np = io.imread("https://img.opf-dev.jp/{}/{}/{}/{}.png".format(img_type, z, x, y))

        # ここで変換処理を噛ませる
        # hoge()

        # 表示のために型変換
        img_bytes = bytes_io.BytesIO() #空のインスタンスを作る
        io.imsave(img_bytes, img_np)
        img_base64 = base64.b64encode(img_bytes.getvalue()).decode('utf-8')

        # 出力
        print('<img src="data:image/png;base64, {}" />'.format(img_base64))
    except:
        print("ERROR: Image not found")

ERROR: Input format error. please input `/img/[img_type]/[z]/[x]/[y]`
ex. http://localhost:8822/img/osm/13/7252/3234


# NDSI画像を取得するためのAPI

In [8]:
def ditect_snow(img):
    # グレイ変換
    img_gray_01 = color.rgb2gray(img)
    # 値のレンジを変更(0-1 -> 0-255)
    img_gray = img_as_ubyte(img_gray_01)

    # しきい値処理
    height, width= img_gray.shape
    img_binary = np.zeros((height, width)) # 初期化
    img_binary = img_gray < 230 # しきい値にもとづいて，2値化処理実行

    #雪だけをマスキング
    img_msk_rev = img.copy()
    img_msk_rev[~img_binary] = 40
    return img_msk_rev, img_binary

In [9]:
def ditect_ndsi(img_band2, img_band4):
    # NDSI（Normalized Difference Snow Index）を計算
    img_NDSI = (img_band2 - img_band4) / (img_band2 + img_band4)  # band4ではなく5を使わなくてはいけない？

    # 最大最小を制限
    img_NDSI = np.clip(img_NDSI, 0, 1)

    return img_NDSI

In [16]:
# GET /ndsi_img/:z/:x/:y

# 読み取り
request = json.loads(REQUEST)

# 必要項目抽出
x = request['path'].get('x')
y = request['path'].get('y')
z = request['path'].get('z')
# z="13"
# x="7252"
# y="3234"

if validation_check(x, y, z):
    try:
        urls=["{}/{}/{}/{}/{}.png".format(URL_DOMAIN, img_type, z, x, y) for img_type in img_types]
        imgs = np.array([io.imread(url) for url in urls])

        band2 = imgs[6]
        band4 = imgs[8]

        # 変換処理
        img_msk_rev, img_binary = ditect_snow(imgs[1])

        # #雪以外をマスキングした場合
        img_NDSI = ditect_ndsi(band2, band4)
        img_NDSI[img_binary] = 0

        # 表示のために型変換
        img_bytes = bytes_io.BytesIO() #空のインスタンスを作る
        io.imsave(img_bytes, img_NDSI)
        img_base64 = base64.b64encode(img_bytes.getvalue()).decode('utf-8')

        # 出力
        print('<img src="data:image/png;base64, {}" />'.format(img_base64))
    except:
        print("ERROR: Image not found")

ERROR: Input format error. please input `/img/[img_type]/[z]/[x]/[y]`
ex. http://localhost:8822/img/osm/13/7252/3234
